In [1]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
import sys, os
fn_data = "../data/renewables_with_load.parquet"
fn_cap = "../data/renewables_capacity.parquet"
from data import get_generation, normalize_generation, get_profiles
from graphs import plot_profile, plot_daily_generation

2024-03-21 11:25:56.479 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.482 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.484 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.486 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.488 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.490 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.492 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:56.494 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-

In [2]:
df_gen, df_annual = get_generation(fn_data, fn_cap, country="DE", year=2023)
df_norm = normalize_generation(df_gen, shares={"Wind": 0.4, "Solar": 0.4, "Baseload": 0.2}, total_demand=0)
profiles = get_profiles(df_norm)

2024-03-21 11:25:56.919 
  command:

    streamlit run e:\GIT\baseload\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-03-21 11:25:56.920 No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:57.213 No runtime found, using MemoryCacheStorageManager
2024-03-21 11:25:57.225 No runtime found, using MemoryCacheStorageManager


!!!!!!!!!!!!!!


       solar  windOffshore  windOnshore
0  63065.55       8128.82     57589.83


In [ ]:
df = 

In [3]:
profiles

{'Hourly: Year':               Wind         Solar      Baseload        Demand
 Hour                                                        
 0     21704.616694      0.000000  10427.226485  41775.432164
 1     21450.112893      0.000000  10427.226485  41421.215432
 2     21226.212451      1.978346  10427.226485  41852.840212
 3     21157.611673    226.388779  10427.226485  43542.830685
 4     21229.976350   2125.928220  10427.226485  47406.574452
 5     20873.557543   7863.153114  10427.226485  51757.933479
 6     20114.643765  18697.663892  10427.226485  55245.846712
 7     19551.297495  33220.652274  10427.226485  57249.540377
 8     19290.414454  48062.464694  10427.226485  58427.058260
 9     19429.819882  59257.010803  10427.226485  59359.867616
 10    19653.715602  65182.100678  10427.226485  59403.832473
 11    19781.654572  65906.136843  10427.226485  58520.157945
 12    19911.313855  61770.746336  10427.226485  57231.267959
 13    20118.060743  53030.840092  10427.226485  56191

In [3]:
def get_storage_stats(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Get curtailment and energy overshoot given the frame of generation

    Args:
        df: frame with hourly generation and demand data

    Returns:
        hourly dataframe with storage statistics; aggregated statistics
    """
    col_gen = [c for c in df.columns if c != "Demand"]
    df_ = df.assign(
        TotalSupply=lambda df: df[col_gen].sum(1),
        ExcessSupply=lambda df: (df["TotalSupply"] - df["Demand"]).map(
            lambda x: max(x, 0)
        ),
        ExcessDemand=lambda df: (df["Demand"] - df["TotalSupply"]).map(
            lambda x: max(x, 0)
        ),
    )
    total = df_.sum()
    stats = {
        var: {
            "Total": total[var],
            "TotalPercentOfDemand": total[var] / total["Demand"] * 100,
            "Max": df_[var].max(),
            "Hours": len(df_[df_[var] > 9]),
        }
        for var in ["ExcessSupply", "ExcessDemand"]
    }
    return df_, pd.DataFrame.from_dict(stats).round(1)

df, stats = get_storage_stats(df_norm)
stats
df_s = df.iloc[:24,:][["ExcessSupply", "ExcessDemand"]].copy()


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2023-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Wind          8760 non-null   float64
 1   Solar         8760 non-null   float64
 2   Baseload      8760 non-null   float64
 3   Demand        8760 non-null   float64
 4   TotalSupply   8760 non-null   float64
 5   ExcessSupply  8760 non-null   float64
 6   ExcessDemand  8760 non-null   float64
dtypes: float64(7)
memory usage: 547.5 KB


In [5]:
df_norm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2023-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Wind      8760 non-null   float64
 1   Solar     8760 non-null   float64
 2   Baseload  8760 non-null   float64
 3   Demand    8760 non-null   float64
dtypes: float64(4)
memory usage: 342.2 KB


In [6]:
import numpy as np

In [7]:
# objective: simply minimize the sum
c = np.ones((1, len(df_s)))

# build the constraint matrix
df_s
c

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]])